In [1]:
!pip install -q -U kaggle_environments

In [3]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

In [4]:
import math


def get_score(left_move, right_move):
    # This method exists in this file so it can be consumed from rps.py and agents.py without a circular dependency
    delta = (
        right_move - left_move
        if (left_move + right_move) % 2 == 0
        else left_move - right_move
    )
    return 0 if delta == 0 else math.copysign(1, delta)

1. Опишем поведение агента, всегда играющего "камень" - это значение 0

In [5]:
def rock_agent(observation, configuration):
    return 0

2. Попробуем теперь использовать информацию о прошлых действиях противника. Опишем агента, который производит то же самое действие, что и оппонент на прошлом ходу

In [6]:
def copy_opponent(observation, configuration):
    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, configuration.signs)

3. Ножницы

In [7]:
def scissors_agent(observation, configuration):
    # 2 - scissors
    return 2

4. Бумага

In [8]:
def paper_agent(observation, configuration):
    #1 - paper
    return 1

5. Ход, который нужен был для победы в прошлой игре

In [9]:
last_react_action = None

def reactionary_agent(observation, configuration):
    global last_react_action
    if observation.step == 0:
        last_react_action = random.randrange(0, configuration.signs)
    elif get_score(last_react_action, observation.lastOpponentAction) <= 1:
        last_react_action = (observation.lastOpponentAction + 1) % configuration.signs

    return last_react_action

6.

In [10]:
def contr_react_agent(observation, configuration):
    # инициализируем переменную, в которой будем хранить свой последний ход
    last_step = -1
    # если это первый ход
    if last_step < 0:
        # записываем в переменную последнего хода 0
        last_step = 0
    # иначе
    else:
        # записываем в переменную последнего хода значение на 2 больше,
        # чем последний ход соперника
        last_step = (last_step + 2) % configuration.signs
    # возвращаем получившееся значение
    return last_step

7. Циклический агент. Ходит по кругу 0-1-2-0-1-2

In [11]:
def mono_agent(observation, configuration):
    # если это первый ход
    if observation.step == 0:
        # возвращаем рандомное значение из доступных вариантов
        return random.randrange(0, configuration.signs)
    # иначе берем остаток от деления номера текущего эпизода на
    # количество возможных вариантов
    return observation.step % configuration.signs

8. Статистичекий агент

In [12]:
action_histogram = {}


def statistical(observation, configuration):
    global action_histogram
    if observation.step == 0:
        # сбрасываем статистику
        action_histogram = {}
        # возвращаем рандомное значение из доступных вариантов
        return random.randrange(0, configuration.signs)
    # сохраняем в переменную последний ход соперника
    action = observation.lastOpponentAction
    # если такого ключа еще нет в справочнике
    if action not in action_histogram:
        # добавляем в справочник со значением 0
        action_histogram[action] = 0
    # делаем инкремент счетчика для такого хода соперника
    action_histogram[action] += 1

    # определяем статистически самый частый ход соперника
    # инициализируем переменную для хранения хода с максимальным счетчиком
    mode_action = None
    # инициализируем переменную для хранения максимального значения счетчика
    mode_action_count = None
    # для каждой пары ключ-значение в справочнике статистики
    for k, v in action_histogram.items():
        # если значение в справочнике для этого ключа не установлено
        # или значение итератора больше значения в справочнике
        if mode_action_count is None or v > mode_action_count:
            # сохраняем текущий ключ в переменную максимума
            mode_action = k
            # сохраняем текущее значение в переменную максимума
            mode_action_count = v
            # переходим к следующей итерации
            continue
    # ходим так, чтобы победить самый частый ход соперника
    return (mode_action + 1) % configuration.signs

9. Рандомный агент. Ходит всегда случайнр

In [13]:
def rand_agent(observation, configuration):
    # возвращаем рандомное значение из доступных вариантов
    return random.randrange(0, configuration.signs)

10. Циклический агент со смещением в 1 шаг. 0-2-1-0-2-1

In [14]:
def mono_by_one_step(observation, configuration):
    # инициализируем поле для хранения последнего хода
    last_step = -1
    # если это первый ход
    if last_step < 0:
        # возвращаем рандомное значение из доступных вариантов
        last_step = random.randrange(0, configuration.signs)
    # прибавляем 2 к своему последнему ходу
    return (last_step + 2) % configuration.signs


11. Циклический агент, который ходит 2 раза подряд один шаг 0-0-1-1-2-2

In [15]:
def mono_two_times_step(observation, configuration):
    # инициализируем поле для хранения последнего хода
    last_step = -1
    # инициализируем поле для хранения  количества повторов
    repeats = 0

    if last_step < 0:
        # возвращаем рандомное значение из доступных вариантов
        last_step = random.randrange(0, configuration.signs)
    # если повтора еще не было
    if repeats == 0:
        # инкрементируем количество повторов
        repeats += 1
    # если уже повторяли
    else:
        # сбрасываем счетчик повторов
        repeats = 0
        # инкрементируем значение последнего шага
        last_step = (last_step + 1) % configuration.signs
    # возвращаем значение поля, хранящего последний шаг
    return last_step

In [16]:
def part_agent(observation, configuration):
    # инициализируем поле для хранения последнего хода
    last_step = 0
    # инициализируем поле для хранения  количества повторов
    repeats = 0
    # если количество повторов меньше 1/3 от запланированного количества эпизодов
    if repeats < (configuration.get('episodeSteps') / 3):
        # инкрементируем счетчик повторов
        repeats += 1
    # иначе переход к следующему значению
    else:
        # сбрасываем счетчик повторов
        repeats = 0
        # инкрементируем значение последнего шага
        last_step = (last_step + 1) % configuration.signs
    # возвращаем значение поля, хранящего последний шаг
    return last_step

In [15]:
'''
agents = {
    "rock_agent": rock_agent,
    "copy_opponent": copy_opponent,
    "scissors_agent": scissors_agent,
    "paper_agent": paper_agent,
    "reactionary_agent": reactionary_agent,
    "contr_react_agent": contr_react_agent,
    "mono_agent": mono_agent,
    "statistical": statistical,
    "rand_agent": rand_agent,
    "mono_by_one_step": mono_by_one_step,
    "mono_two_times_step": mono_two_times_step
}
'''

In [75]:
agents_list = ['rock_agent',
               'copy_opponent',
               'scissors_agent',
               'paper_agent',
               'reactionary_agent',
               'contr_react_agent',
               'mono_agent',
               'statistical',
               'rand_agent',
               'mono_by_one_step',
               'mono_two_times_step',
               'part_agent']

In [47]:
import random

agents_list = [rock_agent, copy_opponent, scissors_agent, paper_agent, reactionary_agent, contr_react_agent, mono_agent, statistical, rand_agent, mono_by_one_step, mono_two_times_step, part_agent]


def simulate_match(team1, team2):
    """Симулирует матч между двумя командами и возвращает победителя."""

    res = evaluate(
        "rps", #environment to use - no need to change
        [team1, team2], #agents to evaluate
        configuration={"episodeSteps": 2000}, #number of episodes,
    )

    if res[0][0] > res[0][1]:
        return team1
    else:
        return team2


def conduct_tournament(agents_list):
    """Проводит турнир с выбыванием и возвращает победителя."""


    round_number = 1
    while len(agents_list) > 1:
        print(f"Раунд {round_number}:")
        next_round_teams = []
        # Если команд нечетное количество, одна из них проходит дальше автоматически
        if len(agents_list) % 2 == 1:
            bye_team = agents_list.pop()
            next_round_teams.append(bye_team)
            print(f"Команда {bye_team} проходит в следующий раунд автоматически")

        random.shuffle(agents_list)
        # Проводим матчи
        for i in range(0, len(agents_list), 2):
            team1 = agents_list[i]
            team2 = agents_list[i + 1]
            winner = simulate_match(team1, team2)
            next_round_teams.append(winner)
            print(f"Матч: {team1} vs {team2}, Победитель: {winner}")

        agents_list = next_round_teams
        round_number += 1
        print()  # Пустая строка для разделения раундов

    print(f"Победитель турнира: {agents_list[0]}")
    return agents_list[0]

# Пример использования

winner = conduct_tournament(agents_list)

Раунд 1:
Матч: <function rock_agent at 0x7ce3787175b0> vs <function scissors_agent at 0x7ce378717880>, Победитель: <function rock_agent at 0x7ce3787175b0>
Матч: <function mono_by_one_step at 0x7ce378717760> vs <function part_agent at 0x7ce378717d90>, Победитель: <function part_agent at 0x7ce378717d90>
Матч: <function mono_agent at 0x7ce3787179a0> vs <function mono_two_times_step at 0x7ce378717be0>, Победитель: <function mono_two_times_step at 0x7ce378717be0>
Матч: <function copy_opponent at 0x7ce4adabdb40> vs <function reactionary_agent at 0x7ce3787176d0>, Победитель: <function reactionary_agent at 0x7ce3787176d0>
Матч: <function paper_agent at 0x7ce378717910> vs <function statistical at 0x7ce378717ac0>, Победитель: <function statistical at 0x7ce378717ac0>
Матч: <function rand_agent at 0x7ce378717c70> vs <function contr_react_agent at 0x7ce3787177f0>, Победитель: <function contr_react_agent at 0x7ce3787177f0>

Раунд 2:
Матч: <function mono_two_times_step at 0x7ce378717be0> vs <function

Воспользуемся функцией evaluate из библиотеки kaggle_environments с помощью которой запустим наших агентов и проведем эксперимент на заданном количестве игр

In [ ]:
evaluate(
    "rps", #environment to use - no need to change
    [mono_two_times_step, mono_by_one_step], #agents to evaluate
    configuration={"episodeSteps": 10000} #number of episodes
)

[[100.0, -100.0]]

In [43]:
evaluate(
    "rps", #environment to use - no need to change
    [copy_opponent, part_agent], #agents to evaluate
    configuration={"episodeSteps": 2000} #number of episodes
)

[[0, 0]]

In [ ]:
agents_list = ['rock_agent',
               'copy_opponent',
               'scissors_agent',
               'paper_agent',
               'reactionary_agent',
               'contr_react_agent',
               'mono_agent',
               'statistical',
               'rand_agent',
               'mono_by_one_step',
               'mono_two_times_step',
               'part_agent']